In [14]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("odometer analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [15]:
spark.sparkContext.setLogLevel("INFO") 

In [16]:

df = spark.read.load("/Users/rc390/Downloads/dv_data/01/ingestion-transformed-data-2018-04-01-14-05-39-297-62.csv",
                     format="csv", sep="\t", inferSchema="true", header="false")

In [17]:
df.printSchema()

root
 |-- _c0: timestamp (nullable = true)
 |-- _c1: long (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)



In [18]:
from pyspark.sql.functions import substring
from pyspark.sql.types import TimestampType, DateType

In [19]:

df = df.filter(df['_c2'] == 'odometer')
df2 = df.withColumn('timestamp', (df['_c1']/1000).cast(TimestampType()))
df2.printSchema()
df2.first()

root
 |-- _c0: timestamp (nullable = true)
 |-- _c1: long (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



Row(_c0=datetime.datetime(2018, 3, 31, 0, 0), _c1=1522522999367, _c2='odometer', _c3='WDDUX6EB5JA', _c4='{"value":9639.4}', _c5='2973cf9c-a61f-408e-a350-bf0e93dc2303', _c6=78, _c7=47, _c8=None, _c9='WDDUX6EB5JA354527', timestamp=datetime.datetime(2018, 4, 1, 3, 3, 19, 367000))

In [20]:
from pyspark.sql.functions import to_date, hour

In [21]:
df3 = df2.withColumn('hour', hour(df2.timestamp))

In [22]:
df3 = df3.withColumn('date', df3.timestamp.cast(DateType()))

In [23]:
df3.first()

Row(_c0=datetime.datetime(2018, 3, 31, 0, 0), _c1=1522522999367, _c2='odometer', _c3='WDDUX6EB5JA', _c4='{"value":9639.4}', _c5='2973cf9c-a61f-408e-a350-bf0e93dc2303', _c6=78, _c7=47, _c8=None, _c9='WDDUX6EB5JA354527', timestamp=datetime.datetime(2018, 4, 1, 3, 3, 19, 367000), hour=3, date=datetime.date(2018, 4, 1))

In [24]:
from pyspark.sql import functions as F
df3.groupBy(['_c9', 'date', 'hour']).agg(F.max('_c1')-F.min('_c1')).collect()

[Row(_c9='WDDUG6CB9JA382432', date=datetime.date(2018, 4, 1), hour=6, (max(_c1) - min(_c1))=0),
 Row(_c9='LE4ZG4JB6JL129199', date=datetime.date(2018, 4, 1), hour=6, (max(_c1) - min(_c1))=0),
 Row(_c9='WDDUG6CB1JA373143', date=datetime.date(2018, 4, 1), hour=7, (max(_c1) - min(_c1))=0),
 Row(_c9='WDDUX6EB7JA355940', date=datetime.date(2018, 4, 1), hour=7, (max(_c1) - min(_c1))=0),
 Row(_c9='WDDUG5FB9JA345399', date=datetime.date(2018, 4, 1), hour=7, (max(_c1) - min(_c1))=2399859),
 Row(_c9='WDDUG6CB6JA348996', date=datetime.date(2018, 4, 1), hour=7, (max(_c1) - min(_c1))=0),
 Row(_c9='WDDUG6EB7JA380692', date=datetime.date(2018, 4, 1), hour=8, (max(_c1) - min(_c1))=3000000),
 Row(_c9='WDDUG6EB3JA371178', date=datetime.date(2018, 4, 1), hour=8, (max(_c1) - min(_c1))=600002),
 Row(_c9='WDDUG6CB4JA372522', date=datetime.date(2018, 4, 1), hour=8, (max(_c1) - min(_c1))=1800833),
 Row(_c9='WDDUG6CB6JA357679', date=datetime.date(2018, 4, 1), hour=8, (max(_c1) - min(_c1))=0),
 Row(_c9='WDDUG6E

In [13]:
spark.stop()